In [1]:
# mongoimport --db giw --collection pedidos --file pedidos.json
# mongoimport --db giw --collection usuarios --file usuarios.json

from pymongo import MongoClient

mongoClient = MongoClient()
db = mongoClient['giw']

#**********************************************************************************************************************

'''

# CONSULTA 1

collection = db['usuarios'];

n = 1;

result = collection.aggregate([
    { "$group": { "_id": "$pais" , "usuarios": { "$sum": 1  }  } } , 
    { "$project": { "_id": 0, "pais": "$_id", "usuarios": "$usuarios" } } ,
    { "$sort": { "usuarios": -1 , "_id": 1  } } , 
    { "$limit": n }
]);

for row in result:
    
    print(row);

'''

#**********************************************************************************************************************

'''
# CONSULTA 2

collection = db['pedidos'];

m = 2.34; 

result = collection.aggregate([
    { "$unwind": "$lineas" } ,
    { "$match": { "lineas.precio": { "$gte": m } } } , 
    { "$group": { "_id": { "nombre": "$lineas.nombre" , "precio": "$lineas.precio" } , "unidades": { "$sum": 1 }  } } , 
    { "$project": { "_id": 0 , "nombre": "$_id.nombre" , "unidades": "$unidades" , "precio": "$_id.precio"  } } 
]);

for row in result:
    
    print(row);
'''

#**********************************************************************************************************************

'''
# CONSULTA 3

collection = db['usuarios'];

n = 81;

result = collection.aggregate([
{ "$group": { "_id": "$pais" , "mp": { "$min": "$edad" } , "mm": { "$max": "$edad" } , "usuarios": { "$sum": 1 } } } ,                          
{ "$match": { "usuarios": { "$gt": n } } } ,
{ "$project": { "_id": 0 , "pais": "$_id", "rango": { "$subtract": ["$mm","$mp"] } } } ,
{ "$sort": { "rango": -1 , "pais": 1 } }
]);

for row in result:
    
    print(row);
'''

#**********************************************************************************************************************

'''
# CONSULTA 4

collection = db['usuarios'];

result = collection.aggregate([
    { "$lookup": { "from": "pedidos", "localField": "_id", "foreignField": "cliente", "as": "pedidosUsuario" } } ,
    { "$unwind": "$pedidosUsuario" } , { "$project": { "_id": 1, "pais": 1, "totalLineasPedido": { "$size": "$pedidosUsuario.lineas" } } } ,
    { "$group": { "_id": "$pais", "totalPedidos": { "$sum": 1 } , "totalLineas": { "$sum": "$totalLineasPedido" } } } ,                                               
    { "$project": { "_id": 0, "pais": "$_id", "numeroMedioDeLineas": { "$divide": ["$totalLineas","$totalPedidos"] } } } 
]);

for row in result:
    
    print(row);
'''

#**********************************************************************************************************************

'''

# CONSULTA 5

collection = db['usuarios'];

c = "Canada";

result = collection.aggregate([
    { "$match": { "pais": { "$eq": c } } } ,
    { "$lookup": { "from": "pedidos", "localField": "_id", "foreignField": "cliente", "as": "pedidosUsuario" } } ,
    { "$unwind": "$pedidosUsuario" } ,
    { "$project": { "_id": 0, "pais": 1, "total": "$pedidosUsuario.total" } } ,
    { "$group": { "_id": "$pais", "totalPedidos": { "$sum": "$total" } } } ,
    { "$project": { "_id": 0, "pais": "$_id", "totalPedidos": "$totalPedidos" } }
]);

for row in result:
    
    print(row);

'''


'\n\n# CONSULTA 5\n\ncollection = db[\'usuarios\'];\n\nc = "Canada";\n\nresult = collection.aggregate([\n    { "$match": { "pais": { "$eq": c } } } ,\n    { "$lookup": { "from": "pedidos", "localField": "_id", "foreignField": "cliente", "as": "pedidosUsuario" } } ,\n    { "$unwind": "$pedidosUsuario" } ,\n    { "$project": { "_id": 0, "pais": 1, "total": "$pedidosUsuario.total" } } ,\n    { "$group": { "_id": "$pais", "totalPedidos": { "$sum": "$total" } } } ,\n    { "$project": { "_id": 0, "pais": "$_id", "totalPedidos": "$totalPedidos" } }\n]);\n\nfor row in result:\n    \n    print(row);\n\n'

In [2]:
# CONSULTA 2

collection = db['pedidos'];

m = 2.34; 

result = collection.aggregate([
    { "$unwind": "$lineas" } ,
    { "$match": { "lineas.precio": { "$gte": m } } } , 
    { "$group": { "_id": { "nombre": "$lineas.nombre" , "precio": "$lineas.precio" } , "unidades": { "$sum": 1 }  } } , 
    { "$project": { "_id": 0 , "nombre": "$_id.nombre" , "unidades": "$unidades" , "precio": "$_id.precio"  } } 
]);

for row in result:
    
    print(row);

{'nombre': 'velit', 'unidades': 1325, 'precio': 3.44}
{'nombre': 'delectus', 'unidades': 1318, 'precio': 4.34}
{'nombre': 'autem', 'unidades': 1308, 'precio': 5.27}
{'nombre': 'ducimus', 'unidades': 1355, 'precio': 3.8}
{'nombre': 'quis', 'unidades': 1324, 'precio': 3.34}
{'nombre': 'mollitia', 'unidades': 1334, 'precio': 3.62}
{'nombre': 'ea', 'unidades': 1318, 'precio': 2.44}
{'nombre': 'est', 'unidades': 1322, 'precio': 5.59}
{'nombre': 'iusto', 'unidades': 1341, 'precio': 4.48}
{'nombre': 'id', 'unidades': 1311, 'precio': 3.63}
{'nombre': 'veritatis', 'unidades': 1325, 'precio': 4.55}
{'nombre': 'nihil', 'unidades': 1294, 'precio': 2.73}
{'nombre': 'ab', 'unidades': 1357, 'precio': 3.43}
{'nombre': 'quae', 'unidades': 1318, 'precio': 4.04}
{'nombre': 'voluptatibus', 'unidades': 1378, 'precio': 3.74}
{'nombre': 'numquam', 'unidades': 1360, 'precio': 3.2}
{'nombre': 'possimus', 'unidades': 1293, 'precio': 4.03}
